In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset
data = pd.read_csv("C:/Users/user/Downloads/Ziadoo/FER Dataset/fer2013.csv")

# Filter out rows where 'emotion' column has values 0 or 4
filtered_data = data[~data['emotion'].isin([4])]

# Define a mapping of old values to new values
replacement_mapping = {5: 4, 6: 5}

# Replace the 'emotion' column values based on the mapping, using .loc to avoid SettingWithCopyWarning
filtered_data.loc[:, 'emotion'] = filtered_data['emotion'].replace(replacement_mapping)

# Split the dataset based on the type
training_data = filtered_data[filtered_data.iloc[:, 2] == 'Training']
public_test_data = filtered_data[filtered_data.iloc[:, 2] == 'PublicTest']
private_test_data = filtered_data[filtered_data.iloc[:, 2] == 'PrivateTest']
test_data = filtered_data[(filtered_data.iloc[:, 2] == 'PublicTest') | (filtered_data.iloc[:, 2] == 'PrivateTest')]

In [3]:
# Preprocess the pixel data
def preprocess_data(data):
    # Convert pixels from string to numpy arrays
    images = data.iloc[:, 1].apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape(48,48))
    
    # Stack images and expand dimensions to make them compatible with VGG16 (3-channel RGB)
    images = np.stack(images, axis=0).reshape(-1, 48, 48, 1)
    images = np.repeat(images, 3, axis=3)
    
    # Normalize the images
    images = images / 255.0
    
    # Convert labels to numpy array
    labels = data.iloc[:, 0].values
    
    return images, labels

# Apply preprocessing to each data split
train_images, train_labels = preprocess_data(training_data)
public_test_images, public_test_labels = preprocess_data(public_test_data)
private_test_images, private_test_labels = preprocess_data(private_test_data)
test_images, test_labels = preprocess_data(test_data)

In [4]:
# Convert to torch tensors
train_images = torch.tensor(train_images).permute(0, 3, 1, 2).float().to(device)
train_labels = torch.tensor(train_labels).long().to(device)
public_test_images = torch.tensor(public_test_images).permute(0, 3, 1, 2).float().to(device)
public_test_labels = torch.tensor(public_test_labels).long().to(device)
private_test_images = torch.tensor(private_test_images).permute(0, 3, 1, 2).float().to(device)
private_test_labels = torch.tensor(private_test_labels).long().to(device)
test_images = torch.tensor(test_images).permute(0, 3, 1, 2).float().to(device)
test_labels = torch.tensor(test_labels).long().to(device)

In [5]:
# Create TensorDatasets
train_dataset = TensorDataset(train_images, train_labels)
public_test_dataset = TensorDataset(public_test_images, public_test_labels)
private_test_dataset = TensorDataset(private_test_images, private_test_labels)
test_dataset = TensorDataset(test_images, test_labels)

In [6]:
# Create DataLoaders
batch_size = 32  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
public_test_loader = DataLoader(public_test_dataset, batch_size=batch_size, shuffle=False)
private_test_loader = DataLoader(private_test_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Modify VGG16
vgg16 = models.vgg16(pretrained=False)  # Set to False since we're training from scratch
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 6)  # Modify the last layer
vgg16 = vgg16.to(device)

C:\Users\user\miniconda3\envs\newcuda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\miniconda3\envs\newcuda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
def train_model(model, train_loader, validation_loader, device, epochs):
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer (you can change the learning rate if needed)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
    
    # Scheduler for learning rate decay
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = validation_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    print('Training complete')

    return model

In [9]:
# Split training data into training and validation sets
train_data, val_data = train_test_split(train_dataset, test_size=0.1, random_state=42)

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Train the model
trained_model = train_model(vgg16, train_loader, val_loader, device, epochs=50)

Epoch 1/50
----------
Train Loss: 1.6401 Acc: 0.3008
Val Loss: 1.6312 Acc: 0.2986

Epoch 2/50
----------
Train Loss: 1.5525 Acc: 0.3487
Val Loss: 1.3984 Acc: 0.4338

Epoch 3/50
----------
Train Loss: 1.3329 Acc: 0.4665
Val Loss: 1.2348 Acc: 0.5142

Epoch 4/50
----------
Train Loss: 1.2030 Acc: 0.5224
Val Loss: 1.1687 Acc: 0.5389

Epoch 5/50
----------
Train Loss: 1.1036 Acc: 0.5730
Val Loss: 1.1124 Acc: 0.5833

Epoch 6/50
----------
Train Loss: 1.0238 Acc: 0.6036
Val Loss: 1.0569 Acc: 0.5905

Epoch 7/50
----------
Train Loss: 0.9535 Acc: 0.6323
Val Loss: 1.0413 Acc: 0.6022

Epoch 8/50
----------
Train Loss: 0.7380 Acc: 0.7256
Val Loss: 0.9467 Acc: 0.6495

Epoch 9/50
----------
Train Loss: 0.6324 Acc: 0.7684
Val Loss: 0.9745 Acc: 0.6579

Epoch 10/50
----------
Train Loss: 0.5297 Acc: 0.8083
Val Loss: 1.0108 Acc: 0.6642

Epoch 11/50
----------
Train Loss: 0.4138 Acc: 0.8563
Val Loss: 1.1383 Acc: 0.6671

Epoch 12/50
----------
Train Loss: 0.2983 Acc: 0.8997
Val Loss: 1.2973 Acc: 0.6487

E

In [10]:
def test_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    print('Confusion Matrix:')
    print(conf_matrix)

In [11]:
# Test the model
print("Testing on Public Test Loader")
test_model(trained_model, public_test_loader, device)

Testing on Public Test Loader
Accuracy: 66.83%
Confusion Matrix:
[[275   9  79  30   6  68]
 [ 18  24   9   2   0   3]
 [ 75   4 258  38  25  96]
 [ 41   0  38 730  19  67]
 [ 14   1  69  19 296  16]
 [ 57   2 100  66   3 379]]


In [12]:
print("\nTesting on Private Test Loader")
test_model(trained_model, private_test_loader, device)


Testing on Private Test Loader
Accuracy: 65.88%
Confusion Matrix:
[[265   6 109  38   3  70]
 [ 14  29   8   2   0   2]
 [ 84  10 267  36  46  85]
 [ 38   2  38 712  19  70]
 [  8   2  80  19 287  20]
 [ 67   3  78  62   3 413]]


In [13]:
print("\nTesting on Combined Test Set")
test_model(trained_model, test_loader, device)


Testing on Combined Test Set
Accuracy: 66.35%
Confusion Matrix:
[[ 540   15  188   68    9  138]
 [  32   53   17    4    0    5]
 [ 159   14  525   74   71  181]
 [  79    2   76 1442   38  137]
 [  22    3  149   38  583   36]
 [ 124    5  178  128    6  792]]
